In [1]:
# Import required libraries

import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.regularizers import L2
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.model_selection import train_test_split
import logging
logging.getLogger("tensorflow").setLevel(logging.ERROR)
tf.autograph.set_verbosity(0)

2024-04-20 22:27:43.682183: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2 AVX AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [17]:
# Data preprocessing

def get_data_preprocessed(df):
    df = df.drop(columns=['PassengerId', 'Name', 'Ticket', 'Cabin', 'Embarked'])
    df = pd.get_dummies(df, columns=['Sex'], drop_first=True)
    df.head()
    X = df.copy()
    return X

In [18]:
# Split train.csv in train, test dataset

df = pd.read_csv('train.csv')
X = get_data_preprocessed(df)
X = X.drop('Survived', axis=1).astype(float)
y = df[['Survived']].astype(float)
X_train, X_cv, y_train, y_cv = train_test_split(X, y, test_size=0.2)

In [19]:
# Prepare model using neural network

def prepare_model(X_train, y_train):
    model = Sequential([
        tf.keras.Input(shape=(6,)),
        Dense(units=24, activation='relu'),
        Dense(units=12, activation='relu'),
        Dense(units=1, activation='sigmoid', kernel_regularizer=L2(0.01))
    ], name='my_model'
    )

    model.compile(
        loss=tf.keras.losses.BinaryCrossentropy(),
        optimizer=tf.keras.optimizers.Adam(0.001),
    )

    model.fit(
        X_train, y_train,
        epochs=100,
        verbose=1
    )
    return model

model = prepare_model(X_train, y_train)

2024-04-20 22:35:03.145090: I tensorflow/core/common_runtime/process_util.cc:146] Creating new thread pool with default inter op setting: 2. Tune using inter_op_parallelism_threads for best performance.


Epoch 1/100
23/23 [==============================] - 1s 7ms/step - loss: 0.7321
Epoch 2/100
23/23 [==============================] - 0s 5ms/step - loss: 0.7137
Epoch 3/100
23/23 [==============================] - 0s 4ms/step - loss: 0.7103
Epoch 4/100
23/23 [==============================] - 0s 4ms/step - loss: 0.7071
Epoch 5/100
23/23 [==============================] - 0s 7ms/step - loss: 0.7044
Epoch 6/100
23/23 [==============================] - 0s 7ms/step - loss: 0.7015
Epoch 7/100
23/23 [==============================] - 0s 5ms/step - loss: 0.6990
Epoch 8/100
23/23 [==============================] - 0s 4ms/step - loss: 0.6968
Epoch 9/100
23/23 [==============================] - 0s 5ms/step - loss: 0.6946
Epoch 10/100
23/23 [==============================] - 0s 6ms/step - loss: 0.6926
Epoch 11/100
23/23 [==============================] - 0s 5ms/step - loss: 0.6907
Epoch 12/100
23/23 [==============================] - 0s 11ms/step - loss: 0.6888
Epoch 13/100
23/23 [================

In [20]:
# Convert the probability predictions to binary predictions based on a threshold (0.5)
y_pred_prob = model.predict(X_cv)
y_pred = (y_pred_prob >= 0.5).astype(int)

# Convert the true labels to int
y_true = y_cv.values.astype(int)

6/6 [==============================] - 0s 10ms/step


In [22]:
# Calculate confusion matrix
conf_matrix = confusion_matrix(y_true, y_pred)

# Print the confusion matrix
print("Confusion Matrix:")
print(conf_matrix)

# Print a classification report for more detailed metrics
report = classification_report(y_true, y_pred)
print("\nClassification Report:")
print(report)

Confusion Matrix:
[[109   0]
 [ 70   0]]

Classification Report:
              precision    recall  f1-score   support

           0       0.61      1.00      0.76       109
           1       0.00      0.00      0.00        70

    accuracy                           0.61       179
   macro avg       0.30      0.50      0.38       179
weighted avg       0.37      0.61      0.46       179



/home/at00808101/anaconda3/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/at00808101/anaconda3/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/at00808101/anaconda3/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [25]:
# Predict using test dataset
test_df = pd.read_csv('test.csv')
X_test = get_data_preprocessed(test_df).astype(float)
y_test_pred_prob = model.predict(X_test)
y_test_pred = (y_test_pred_prob >= 0.5).astype(int)

y_test_pred = pd.DataFrame(y_test_pred, columns=['Survived'])

14/14 [==============================] - 0s 11ms/step


In [31]:
X_test

,Pclass,Age,SibSp,Parch,Fare,Sex_male
0,3.0,34.5,0.0,0.0,7.8292,1.0
1,3.0,47.0,1.0,0.0,7.0000,0.0
2,2.0,62.0,0.0,0.0,9.6875,1.0
3,3.0,27.0,0.0,0.0,8.6625,1.0
4,3.0,22.0,1.0,1.0,12.2875,0.0
...,...,...,...,...,...,...
413,3.0,NaN,0.0,0.0,8.0500,1.0
414,1.0,39.0,0.0,0.0,108.9000,0.0
415,3.0,38.5,0.0,0.0,7.2500,1.0
416,3.0,NaN,0.0,0.0,8.0500,1.0
